In [1]:
import subprocess
import urllib.request
import sys
import time
import ipywidgets as widgets
from IPython.display import display, clear_output
from IPython.core.display import HTML
import json
import subprocess
subprocess.run('pip install google-cloud-compute notebook'.split(" "), text=True, capture_output=True)
import urllib.request
from __future__ import annotations

import sys
from typing import Any

from google.api_core.extended_operation import ExtendedOperation
from google.cloud import compute_v1

# Run the below cell and copy its output to a new cell and run it

In [2]:
%%javascript
// Send data

fetch( 'https://api64.ipify.org/')
  .then(
    response => response.text()
  ).then(
    text => element.append("ip='"+text+"'")
  );

<IPython.core.display.Javascript object>

# Run the below to create a firewall rule to allow access to the GUI

In [4]:
firewall_rule_capture = subprocess.run('gcloud compute firewall-rules list'.split(" "), text=True, capture_output=True) 
rules = firewall_rule_capture.stdout.split("\n")
rule_found = False
for x in rules:
    if 'allow-gui-access' in x:
        rule_found = True
        print("rule already exist")
if not rule_found:
    subprocess.run(['gcloud','compute','firewall-rules','create','allow-gui-access','--allow=tcp:8081','--source-ranges='+ip+'/32','--description="allow-gui-access"'])

rule already exist


# Finally run this block and select the GUI you want

In [5]:
w = widgets.Dropdown(
    options=['bandage', 'pymol', 'molbobity'],
    value='bandage',
    description='GUI:',
)
script_key = 'bandage'
script={
    'bandage':'bandage-gui',
    'pymol':'pymol',
    'molbobity':'molbobity',
}
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        script_key = change['new']
        print("changed to %s" % change['new'])

out = widgets.Output()

# Get the current instances public IP address
url = "http://metadata.google.internal/computeMetadata/v1/instance/name"
req = urllib.request.Request(url)
req.add_header("Metadata-Flavor", "Google")
machine_name = urllib.request.urlopen(req).read().decode()
url = "http://metadata.google.internal/computeMetadata/v1/instance/zone"
req = urllib.request.Request(url)
req.add_header("Metadata-Flavor", "Google")
zone = urllib.request.urlopen(req).read().decode().split("/")[-1]
url = "http://metadata.google.internal/computeMetadata/v1/project/project-id"
req = urllib.request.Request(url)
req.add_header("Metadata-Flavor", "Google")
project_id = urllib.request.urlopen(req).read().decode()
instance_client = compute_v1.InstancesClient()
i = instance_client.get(project=project_id, zone=zone, instance=machine_name)
ip = i.network_interfaces[0].access_configs[0].nat_i_p

# Define a function
def on_button_clicked(b):
    with out:
        script_key = list(script.keys())[w.get_state()['index']]
        
        clear_output()
        docker_out = subprocess.Popen(('docker run --rm -v /home/jupyter:/config/s3 -p 8081:3000 us-east4-docker.pkg.dev/nih-cl-shared-resources/nigms-sandbox/'+script[script_key]).split(" "))
        # if len(server_out.stdout) >0:
        display(HTML("Please access the GUI at <a href=\"http://"+ip+":8081\">http://"+ip+":8081</a><br>Note, it may take 30 seconds for it to fully start "+script_key))
        #     print)
        #     print()
        # else:
        #     display(HTML(server_out.stderr))
        #     print(server_out.stderr)


button = widgets.Button(description="Start Gui")
button.on_click(on_button_clicked)
w.observe(on_change)
display(w)
display(button)
display(out)


Dropdown(description='GUI:', options=('bandage', 'pymol', 'molbobity'), value='bandage')

Button(description='Start Gui', style=ButtonStyle())

Output()